In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import jieba

In [3]:
from self_embedding import get_embedding

### about self：

在Python类中规定，函数的第一个参数是实例对象本身，并且约定俗成，把其名字写为self。其左右相当于java中的this，表示当前类的对象，可以调用当前类中的属性和方法。

对应的self.valueName，self.function()中的valueName和function()具体含义如下：valueName：表示self对象，即实例的变量。与其他的，Class的变量，全局的变量，局部的变量，是相对应的。function：表示是调用的是self对象，即实例的函数。与其他的全局的函数，是相对应的。

## nn.Mudule
torcn.nn是专门为神经网络设计的模块化接口. nn构建于autograd之上，可以用来定义和运行神经网络。

nn.Module是nn中十分重要的类，包含网络各层的定义及forward方法。

nn.Module 有 8 个属性，都是OrderDict(有序字典)。在 LeNet 的__init__()方法中会调用父类nn.Module的__init__()方法，创建这 8 个属性。

_parameters 属性：存储管理 nn.Parameter 类型的参数

_modules 属性：存储管理 nn.Module 类型的参数

_buffers 属性：存储管理缓冲属性，如 BN 层中的 running_mean

5 个 ***_hooks 属性：存储管理钩子函数

## Conv2d
二维卷积可以处理二维数据

nn.Conv2d(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True))
参数：

  in_channel:　输入数据的通道数，例RGB图片通道数为3；
  
  out_channel: 输出数据的通道数，这个根据模型调整；
  
  kennel_size: 卷积核大小，可以是int，或tuple；kennel_size=2,意味着卷积大小(2,2)， kennel_size=（2,3），意味着卷积大小（2，3）即非正方形卷积
  
  stride：步长，默认为1，与kennel_size类似，stride=2,意味着步长上下左右扫描皆为2， stride=（2,3），左右扫描步长为2，上下为3；
  
  padding：　零填充


## nn.Dropout
Dropout是指在模型训练时随机让网络某些隐含层节点的权重不工作，不工作的那些节点可以暂时认为不是网络结构的一部分，但是它的权重得保留下来（只是暂时不更新而已），因为下次样本输入时它可能又得工作了
让某个神经元的激活值以一定的概率p，让其停止工作，这次训练过程中不更新权值，也不参加神经网络的计算。

## nn.Linear
nn.Linear（）是用于设置网络中的全连接层的，需要注意在二维图像处理的任务中，全连接层的输入与输出一般都设置为二维张量，形状通常为[batch_size, size]，不同于卷积层要求输入输出是四维张量。

具体而言：
torch. $\mathrm{nn} .$ Linear(in_features, out_features, bias=True)

in_features指的是输入的二维张量的大小，即输入的[batch_size, size]中的size。

out_features指的是输出的二维张量的大小，即输出的二维张量的形状为[batch_size，output_size]，当然，它也代表了该全连接层的神经元个数。

从输入输出的张量的shape角度来理解，相当于一个输入为[batch_size, in_features]的张量变换成了[batch_size, out_features]的输出张量。

## squeeze函数
torch.squeeze(input, dim=None, out=None)

squeeze()函数的功能是维度压缩。返回一个tensor（张量），其中 input 中大小为1的所有维都已删除。

举个例子：如果 input 的形状为 (A×1×B×C×1×D)，那么返回的tensor的形状则为 (A×B×C×D)

当给定 dim 时，那么只在给定的维度（dimension）上进行压缩操作。

举个例子：如果 input 的形状为 (A×1×B)，squeeze(input, 0)后，返回的tensor不变；squeeze(input, 1)后，返回的tensor将被压缩为 (A×B)

还有 **unsqueeze()** 操作

## max_pool1d

torch.nn.functional.max_pool1d(input, kernel_size, stride=None, padding=0, dilation=1, ceil_mode=False, return_indices=False)

参数：

kernel_size(int or tuple) - max pooling的窗口大小

stride(int or tuple, optional) - max pooling的窗口移动的步长。默认值是kernel_size

padding(int or tuple, optional) - 输入的每一条边补充0的层数

dilation(int or tuple, optional) – 一个控制窗口中元素步幅的参数

return_indices - 如果等于True，会返回输出最大值的序号，对于上采样操作会有帮助

ceil_mode - 如果等于True，计算输出信号大小的时候，会使用向上取整，代替默认的向下取整的操作

shape:

$N$ is batch dimension, $C$ is the channel dimension

输入: ($N$, $C$,$L_{i n}$)

输出: ($N$, $C$,$L_{o u t}$)

$L_{o u t}=\left\lfloor\frac{L_{i n}+2 \times \text { padding }-\text { dilation } \times(\text { kernel_size }-1)-1}{\text { stride }}+1\right\rfloor$

## torch.cat
torch.cat是将两个张量（tensor）拼接在一起，cat是concatenate的意思，即拼接，联系在一起。

C=torch.cat((A,B),0)就表示按维数0（行）拼接A和B，也就是竖着拼接，A上B下。此时需要注意：列数必须一致，即维数1数值要相同，这里都是3列，方能列对齐。拼接后的C的第0维是两个维数0数值和，即2+4=6.

C=torch.cat((A,B),1)就表示按维数1（列）拼接A和B，也就是横着拼接，A左B右。此时需要注意：行数必须一致，即维数0数值要相同，这里都是2行，方能行对齐。拼接后的C的第1维是两个维数1数值和，即3+4=7.

In [4]:
class TextCNN(nn.Module):
    def __init__(self, word_embedding, each_filter_num, filter_heights, drop_out, num_classes):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(word_embedding, freeze = True)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels = 1, out_channels = each_filter_num, kernel_size = (h, word_embedding.shape[1]))for h in filter_heights
            #(h, word_embedding.shape[0])
    
        ])
        
        self.drop_out = nn.Dropout(drop_out)
        self.fc = nn.Linear(each_filter_num * len(filter_heights), num_classes)
        
    def conv_and_pool(self, x, conv):
        
        x = F.relu(conv(x)).squeeze(3)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)

        return x
    
    def forward(self, input_ids = None):
        word_embeddings = self.embedding(input_ids)
        sentence_embedding = word_embeddings.unsqueeze(1)
        
        out = torch.cat([self.conv_and_pool(sentence_embedding, conv) for conv in self.convs], 1)
        out = self.drop_out(out)
        out = self.fc(out)
        
        outputs = (out, )
       
            
        return outputs
        
            

In [5]:
if __name__ == '__main__':
    some_text_sentence = '今天上海大学封闭，股市大跌'
    words = list(jieba.cut(some_text_sentence))
    embedding, token2id, _ = get_embedding(set(words))
    
    text_cnn_model = TextCNN(embedding, each_filter_num = 128, filter_heights = [2, 3, 4], drop_out = 0.3, num_classes = 15)
    ids = [token2id[w] for w in words]
    out = text_cnn_model(torch.tensor([ids])) 
    print(out)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\90392\AppData\Local\Temp\jieba.cache
Loading model cost 0.568 seconds.
Prefix dict has been built successfully.
100%|███████████████████████████████████████████████████████████████████████| 195202/195202 [00:02<00:00, 97162.96it/s]

(tensor([[-0.0428,  0.0379, -0.0926, -0.0111,  0.0021,  0.0827,  0.0329, -0.0138,
          0.1736,  0.0675, -0.0370,  0.0514,  0.0610,  0.1683, -0.0648]],
       grad_fn=<AddmmBackward0>),)


**改了一点参数，也许可以work了。。。**